# Conditional Neural Processes (CNP) for 1D regression.
[Conditional Neural Processes](https://arxiv.org/pdf/1807.01613.pdf) (CNPs) were
introduced as a continuation of
[Generative Query Networks](https://deepmind.com/blog/neural-scene-representation-and-rendering/)
(GQN) to extend its training regime to tasks beyond scene rendering, e.g. to
regression and classification.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import datetime
import numpy as np
import torchsnooper
import os
import plotting_utils as plotting
import data_generator as data
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import dask.dataframe as dd


<img src="../utilities/concept.png" alt="drawing" width="500"/>

## Conditional Neural Processes

We can visualise a forward pass in a CNP as follows:

<img src="https://bit.ly/2OFb6ZK" alt="drawing" width="400"/>

As shown in the diagram, CNPs take in pairs **(x, y)<sub>i</sub>** of context
points, pass them through an **encoder** to obtain
individual representations **r<sub>i</sub>** which are combined using an **aggregator**. The resulting representation **r**
is then combined with the locations of the targets **x<sub>T</sub>** and passed
through a **decoder** that returns a mean estimate
of the **y** value at that target location together with a measure of the
uncertainty over said prediction. Implementing CNPs therefore involves coding up
the three main building blocks:

*   Encoder
*   Aggregator
*   Decoder

A more detailed description of these three parts is presented in the following
sections alongside the code.

## Encoder

The encoder **e** is shared between all the context points and consists of an
MLP with a handful of layers. For this experiment four layers are enough, but we
can still change the number and size of the layers when we build the graph later
on via the variable **`encoder_output_sizes`**. Each of the context pairs **(x,
y)<sub>i</sub>** results in an individual representation **r<sub>i</sub>** after
encoding. These representations are then combined across context points to form
a single representation **r** using the aggregator **a**.

In this implementation we have included the aggregator **a** in the encoder as
we are only taking the mean across all points. The representation **r** produced
by the aggregator contains the information about the underlying unknown function
**f** that is provided by all the context points.

In [ ]:
class DeterministicEncoder(nn.Module):
    def __init__(self, output_sizes):
        super(DeterministicEncoder, self).__init__()
        self.linears = nn.ModuleList()
        for i in range(len(output_sizes) - 1):
            self.linears.append(nn.Linear(output_sizes[i], output_sizes[i + 1]))

    def forward(self, context_x, context_y):
        """Encodes the inputs into one representation.

        Args:
        context_x: Tensor of size of batches x observations x m_ch. For this 1D regression
          task this corresponds to the x-values.
        context_y: Tensor of size bs x observations x d_ch. For this 1D regression
          task this corresponds to the y-values.

        Returns:
            representation: The encoded representation averaged over all context 
            points.
        """

        # Concatenate x and y along the filter axes
        encoder_input = torch.cat((context_x, context_y), dim=-1)

        # Get the shapes of the input and reshape to parallelise across observations
        batch_size, num_context_points, _ = encoder_input.shape
        hidden = encoder_input.view(batch_size * num_context_points, -1)
        
        # Pass through MLP
        for i, linear in enumerate(self.linears[:-1]):
            hidden = torch.relu(linear(hidden))
        # Last layer without a ReLu
        hidden = self.linears[-1](hidden)
        # Bring back into original shape (# Flatten the output feature map into a 1D feature vector)
        hidden = hidden.view(batch_size, num_context_points, -1)

        # Aggregator: take the mean over all points
        representation = hidden.mean(dim=1)
        return representation

## Decoder

Once we have obtained our representation **r** we concatenate it with each of
the targets **x<sub>t</sub>** and pass it through the decoder **d**. As with the
encoder **e**, the decoder **d** is shared between all the target points and
consists of a small MLP with layer sizes defined in **`decoder_output_sizes`**.
The decoder outputs a mean **&mu;<sub>t</sub>** and a variance
**&sigma;<sub>t</sub>** for each of the targets **x<sub>t</sub>**. To train our
CNP we use the log likelihood of the ground truth value **y<sub>t</sub>** under
a Gaussian parametrized by these predicted **&mu;<sub>t</sub>** and
**&sigma;<sub>t</sub>**.

In this implementation we clip the variance **&sigma;<sub>t</sub>** at 0.1 to
avoid collapsing.

In [ ]:
class DeterministicDecoder(nn.Module):
    def __init__(self, output_sizes):
        """CNP decoder.
        Args:
            output_sizes: An iterable containing the output sizes of the decoder MLP.
        """
        super(DeterministicDecoder, self).__init__()
        self.linears = nn.ModuleList()
        for i in range(len(output_sizes) - 1):
            self.linears.append(nn.Linear(output_sizes[i], output_sizes[i + 1]))

    def forward(self, representation, target_x):
        """Decodes the individual targets.

        Args:
            representation: The encoded representation of the context
            target_x: The x locations for the target query

        Returns:
            dist: A multivariate Gaussian over the target points.
            mu: The mean of the multivariate Gaussian.
            sigma: The standard deviation of the multivariate Gaussian.   
        """

        # Get the shapes of the input and reshape to parallelise across observations
        batch_size, num_total_points, _ = target_x.shape
        representation = representation.unsqueeze(1).repeat([1, num_total_points, 1])

        # Concatenate the representation and the target_x
        input = torch.cat((representation, target_x), dim=-1)
        hidden = input.view(batch_size * num_total_points, -1)

        # Pass through MLP
        for i, linear in enumerate(self.linears[:-1]):
            hidden = torch.relu(linear(hidden))
        # Last layer without a ReLu
        hidden = self.linears[-1](hidden)

        # Bring back into original shape
        hidden = hidden.view(batch_size, num_total_points, -1)

        return torch.sigmoid(hidden)

## Model

Now that the main building blocks (encoder, aggregator and decoder) of the CNP
are defined we can put everything together into one model. Fundamentally this
model only needs to include two main methods: 1. A method that returns the log
likelihood of the targets' ground truth values under the predicted
distribution.This method will be called during training as our loss function. 2.
Another method that returns the predicted mean and variance at the target
locations in order to evaluate or query the CNP at test time. This second method
needs to be defined separately as, unlike the method above, it should not depend
on the ground truth target values.

In [ ]:
class DeterministicModel(nn.Module):
    def __init__(self, encoder_sizes, decoder_sizes):
        super(DeterministicModel, self).__init__()
        """Initialises the model.

        Args:
            encoder_output_sizes: An iterable containing the sizes of hidden layers of
                the encoder. The last one is the size of the representation r.
            decoder_output_sizes: An iterable containing the sizes of hidden layers of
                the decoder. The last element should correspond to the dimension of
                the y * 2 (it encodes both mean and variance concatenated)
        """
        self._encoder = DeterministicEncoder(encoder_sizes)
        self._decoder = DeterministicDecoder(decoder_sizes)

    def forward(self, query):
        """Returns the predicted mean and variance at the target points.

        Args:
            query: Array containing ((context_x, context_y), target_x) where:
                context_x: Array of shape batch_size x num_context x 1 contains the 
                    x values of the context points.
                context_y: Array of shape batch_size x num_context x 1 contains the 
                    y values of the context points.
                target_x: Array of shape batch_size x num_target x 1 contains the
                    x values of the target points.
            target_y: The ground truth y values of the target y. An array of 
                shape batchsize x num_targets x 1.

        Returns:
            log_p: The log_probability of the target_y given the predicted
            distribution.
            mu: The mean of the predicted distribution.
            sigma: The variance of the predicted distribution.
        """

        (context_x, context_y), target_x = query
        # Pass query through the encoder and the decoder

        representation = self._encoder(context_x, context_y)
        hidden = self._decoder(representation, target_x)
        
        return hidden

## Running Conditional Neural Processes

Now that we have defined the dataset as well as our model and its components we
can start building everything into the graph. Before we get started we need to
set some variables:

*   **`TRAINING_ITERATIONS`** - a scalar that describes the number of iterations
    for training. At each iteration we will sample a new batch of functions from
    the GP, pick some of the points on the curves as our context points **(x,
    y)<sub>C</sub>** and some points as our target points **(x,
    y)<sub>T</sub>**. We will predict the mean and variance at the target points
    given the context and use the log likelihood of the ground truth targets as
    our loss to update the model.
*   **`MAX_CONTEXT_POINTS`** - a scalar that sets the maximum number of contest
    points used during training. The number of context points will then be a
    value between 3 and `MAX_CONTEXT_POINTS` that is sampled at random for every
    iteration.
*   **`PLOT_AFTER`** - a scalar that regulates how often we plot the
    intermediate results.

In [ ]:
TRAINING_ITERATIONS = int(4000) # Total number of training points: training_iterations * batch_size * max_content_points
#BATCH_SIZE = 100 # number of simulation configurations

MAX_CONTEXT_POINTS = 1000 # 2000 # 4000
MAX_TARGET_POINTS =  2000 # 4000 # 8000
CONTEXT_IS_SUBSET = True
BATCH_SIZE = 1
CONFIG_WISE = False
PLOT_AFTER = int(100)
torch.manual_seed(0)

# all available x config/ physics parameters are ["radius","thickness","npanels","theta","length","height","z_offset","volume","nC_Ge77","time_0[ms]","x_0[m]","y_0[m]","z_0[m]","px_0[m]","py_0[m]","pz_0[m]","ekin_0[eV]","edep_0[eV]","time_t[ms]","x_t[m]","y_t[m]","z_t[m]","px_t[m]","py_t[m]","pz_t[m]","ekin_t[eV]","edep_t[eV]","nsec"]
# Comment: if using data version v1.1 for training, "radius","thickness","npanels","theta","length" is probably necessary
names_x=["radius","thickness","npanels","theta","length","r_0[m]","z_0[m]","time_t[ms]","r_t[m]","z_t[m]","L_t[m]","ln(E0vsET)","edep_t[eV]","nsec"]
name_y ='total_nC_Ge77[cts]'
x_size = len(names_x)
if isinstance(name_y,str):
    y_size = 1
else:
    y_size = len(name_y)

RATIO_TESTING_VS_TRAINING = 1/40
version="v1.2"
path_to_files=f"../simulation/out/LF/{version}/tier2/"
path_out = f'./out/'
f_out = f'{path_out}CNP_{version}_{TRAINING_ITERATIONS}_c{MAX_CONTEXT_POINTS}_t{MAX_TARGET_POINTS}'

Data augmentation methods used:

<img src="../utilities/data_augmentation.png" alt="drawing" width="800"/>

In [ ]:
# Set data augmentation parameters
USE_DATA_AUGMENTATION = "mixup" #"smote" #False #"mixup"
USE_BETA = [0.1,0.1] # uniform => None, beta => [a,b] U-shape [0.5,0.5] Uniform [1.,1.] falling [0.5,2] rising [2,0.5]
SIGNAL_TO_BACKGROUND_RATIO = "" # "_1to4" # used for smote augmentation

if USE_DATA_AUGMENTATION:
    path_out = f'./out/{USE_DATA_AUGMENTATION}/'
    f_out = f'CNP_{version}_{TRAINING_ITERATIONS}_c{MAX_CONTEXT_POINTS}_t{MAX_TARGET_POINTS}_{USE_DATA_AUGMENTATION}{SIGNAL_TO_BACKGROUND_RATIO}'
    if USE_DATA_AUGMENTATION == "mixup":
        path_to_files = f"../simulation/out/LF/{version}/tier3/beta_{USE_BETA[0]}_{USE_BETA[1]}/"
        f_out = f'CNP_{version}_{TRAINING_ITERATIONS}_c{MAX_CONTEXT_POINTS}_t{MAX_TARGET_POINTS}_beta_{USE_BETA[0]}_{USE_BETA[1]}'
    elif USE_DATA_AUGMENTATION == "smote" and CONFIG_WISE == True:
        path_to_files = f"../simulation/out/LF/{version}/tier3/smote{SIGNAL_TO_BACKGROUND_RATIO}/"
        

In [ ]:
# Train dataset
dataset_train = data.DataGeneration(num_iterations=TRAINING_ITERATIONS, num_context_points=MAX_CONTEXT_POINTS, num_target_points=MAX_TARGET_POINTS, batch_size = BATCH_SIZE, config_wise=CONFIG_WISE, path_to_files=path_to_files,x_size=x_size,y_size=y_size, mode = "training", ratio_testing=RATIO_TESTING_VS_TRAINING,sig_bkg_ratio = SIGNAL_TO_BACKGROUND_RATIO, use_data_augmentation=USE_DATA_AUGMENTATION, names_x = names_x, name_y=name_y)
TRAINING_ITERATIONS = dataset_train._num_iterations
dataset_testing = data.DataGeneration(num_iterations=int(np.round(TRAINING_ITERATIONS/PLOT_AFTER))+5, num_context_points=MAX_CONTEXT_POINTS, num_target_points=MAX_TARGET_POINTS, batch_size = 1, config_wise=False, path_to_files=f"../simulation/out/LF/{version}/tier2/",x_size=x_size,y_size=y_size, mode = "testing",ratio_testing=RATIO_TESTING_VS_TRAINING, sig_bkg_ratio = SIGNAL_TO_BACKGROUND_RATIO, use_data_augmentation="None", names_x = names_x, name_y=name_y)
TRAINING_ITERATIONS = dataset_train._num_iterations if TRAINING_ITERATIONS > dataset_train._num_iterations else TRAINING_ITERATIONS
PLOT_AFTER =  int(5 * np.ceil(np.ceil(TRAINING_ITERATIONS/(dataset_testing._num_iterations-2))/5)) if PLOT_AFTER < int(np.ceil(TRAINING_ITERATIONS/(dataset_testing._num_iterations-2))) else PLOT_AFTER


We can now add the model to the graph and finalise it by defining the train step
and the initializer.

In [ ]:

d_x, d_in, representation_size, d_out = x_size , x_size+y_size, 32, y_size
encoder_sizes = [d_in, 32, 64, 128, 128, 128, 64, 48, representation_size]
decoder_sizes = [representation_size + d_x, 32, 64, 128, 128, 128, 64, 48, d_out]

model = DeterministicModel(encoder_sizes, decoder_sizes)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# 

bce = nn.BCELoss()
#kldivloss = nn.KLDivLoss(reduction="batchmean")
#cross_entropy_loss = nn.CrossEntropyLoss()
iter_test = 0
fout = open(f'{path_out}{f_out}_training.txt', "w")

with PdfPages(f'{path_out}{f_out}_training.pdf') as pdf:

    for it in range(TRAINING_ITERATIONS):

        # load data:
        data_train = dataset_train.get_data(it, CONTEXT_IS_SUBSET)

        target_y_cnp = model(data_train.query)
        loss = bce(target_y_cnp,  data_train.target_y)
        loss.backward()

        # Perform gradient descent to update parameters
        optimizer.step()
        
        # reset gradient to 0 on all parameters
        optimizer.zero_grad()

        target_y_cnp = target_y_cnp[0].detach().numpy()
        if it % int(PLOT_AFTER/2) == 0:
            print('{} Iteration: {}/{}, train loss: {}'.format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),it, TRAINING_ITERATIONS,loss))
            fout.write('Iteration: {}/{}, train loss: {}\n'.format(it, TRAINING_ITERATIONS,loss))
        if it % PLOT_AFTER == 0 or it == int(TRAINING_ITERATIONS-1):
            data_test = dataset_testing.get_data(iter_test, CONTEXT_IS_SUBSET)
            target_y_cnp_testing = model(data_test.query)
            test_loss = bce(target_y_cnp_testing,  data_test.target_y)
            
            print("{}, Iteration: {}, test loss: {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), it, test_loss))
            fout.write("{}, Iteration: {}, test loss: {}\n".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), it, test_loss))
            if isinstance(name_y,str):
                fig = plotting.plot(target_y_cnp, data_train.target_y[0].detach().numpy(), f'{loss:.2f}', target_y_cnp_testing[0].detach().numpy(), data_test.target_y[0].detach().numpy(), f'{test_loss:.2f}', it)
            else:
                for k in range(y_size):
                    fig = plotting.plot(target_y_cnp[:,k], data_train.target_y[0].detach().numpy()[:,k], f'{loss:.2f}', target_y_cnp_testing[0].detach().numpy()[:,k], data_test.target_y[0].detach().numpy()[:,k], f'{test_loss:.2f}', it)
            if it % PLOT_AFTER*5 == 0 or it == int(TRAINING_ITERATIONS-1):
                pdf.savefig(fig)
            
            plt.show()
            plt.figure().clear()
            plt.close()
            plt.cla()
            plt.clf()
            iter_test += 1
fout.close()
torch.save(model, path_out)

In [ ]:
# Add further Trainings iterations
ADDITIONAL_ITERATIONS = 0

if ADDITIONAL_ITERATIONS > 0:
    dataset_train = data.DataGeneration(num_iterations=TRAINING_ITERATIONS+ADDITIONAL_ITERATIONS, num_context_points=MAX_CONTEXT_POINTS, num_target_points=MAX_TARGET_POINTS, batch_size = BATCH_SIZE, config_wise=CONFIG_WISE, path_to_files=path_to_files,x_size=x_size,y_size=y_size, mode = "training", ratio_testing=RATIO_TESTING_VS_TRAINING,sig_bkg_ratio = SIGNAL_TO_BACKGROUND_RATIO, use_data_augmentation=USE_DATA_AUGMENTATION, names_x = names_x, name_y=name_y)
    ADDITIONAL_ITERATIONS = dataset_train._num_iterations-TRAINING_ITERATIONS

    fout = open(f'{path_out}{f_out}_training.txt', "w")

    # create a PdfPages object
    pdf = PdfPages(f'{path_out}{f_out}_training.pdf')

    for it in range(TRAINING_ITERATIONS, TRAINING_ITERATIONS+ADDITIONAL_ITERATIONS):
        # load data:
        data_train = dataset_train.get_data(it, CONTEXT_IS_SUBSET)
        # Get the predicted mean and variance at the target points for the testing set
        log_prob, mu, _ = model(data_train.query, data_train.target_y)
        
        # Define the loss
        loss = -log_prob.mean()
        loss.backward()

        # Perform gradient descent to update parameters
        optimizer.step()
        
        # reset gradient to 0 on all parameters
        optimizer.zero_grad()

        if max(mu[0].detach().numpy()) <= 1 and min(mu[0].detach().numpy()) >= 0:
            loss_bce = bce(mu, data_train.target_y)
        else:
            loss_bce = -1.

        mu=mu[0].detach().numpy()
        if it % 100 == 0:
            print('{} Iteration: {}/{}, train loss: {:.4f} (vs BCE {:.4f})'.format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),it, TRAINING_ITERATIONS+ADDITIONAL_ITERATIONS,loss, loss_bce))
            fout.write('Iteration: {}/{}, train loss: {:.4f} (vs BCE {:.4f})\n'.format(it, TRAINING_ITERATIONS+ADDITIONAL_ITERATIONS,loss, loss_bce))
        
        if it % PLOT_AFTER == 0 or it == int(TRAINING_ITERATIONS-1):
            data_testing = dataset_testing.get_data(iter_testing, CONTEXT_IS_SUBSET)
            log_prob_testing, mu_testing, _ = model(data_testing.query, data_testing.target_y)
            # Define the loss
            loss_testing = -log_prob_testing.mean()

            if max(mu_testing[0].detach().numpy()) <= 1 and min(mu_testing[0].detach().numpy()) >= 0:
                loss_bce_testing = bce(mu_testing,  data_testing.target_y)
            else:
                loss_bce_testing = -1.

            mu_testing=mu_testing[0].detach().numpy()
            print("{}, Iteration: {}, test loss: {:.4f} (vs BCE {:.4f})".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), it, loss_testing, loss_bce_testing))
            fout.write("{}, Iteration: {}, test loss: {:.4f} (vs BCE {:.4f})\n".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), it, loss_testing, loss_bce_testing))
            if isinstance(name_y,str):
                fig = plotting.plot(mu, data_train.target_y[0].detach().numpy(), f'{loss:.2f}', mu_testing, data_testing.target_y[0].detach().numpy(), f'{loss_testing:.2f}', it)
            else:
                for k in range(y_size):
                    fig = plotting.plot(mu[:,k], data_train.target_y[0].detach().numpy()[:,k], f'{loss:.2f}', mu_testing[:,k], data_testing.target_y[0].detach().numpy()[:,k], f'{loss_testing:.2f}', it)
            if it % PLOT_AFTER*5 == 0 or it == int(TRAINING_ITERATIONS+ADDITIONAL_ITERATIONS-1):
                pdf.savefig(fig)
                plt.show()
                plt.clf()

            iter_testing += 1
    pdf.close()
    fout.close()

#torch.save(model, path_out)


In [ ]:
#model = torch.load(path_out)

version="v1.2"
mode="LF"
filelist = data.get_all_files(f"../simulation/out/{mode}/{version}/tier2/neutron")
num_total_points = 50000


MAX_CONTEXT_POINTS_NEW = int(1/3 * num_total_points)
MAX_TARGET_POINTS_NEW = 2 * (MAX_CONTEXT_POINTS_NEW)

x = np.empty([0,5])
sum_target_y = np.empty([0,1])
mean_target_y_cnp = np.empty([0,1])

hist_target_sig = hist_target_bkg = hist_pred_sig = hist_pred_bkg = np.zeros(100)
fout = open(f'{path_out}{f_out}_training.txt', "a")

pdf = PdfPages(f'{path_out}{f_out}_result.pdf')
for i,file in enumerate(filelist):
    path_to_files = file[:-4]
    dataset_config = data.DataGeneration(num_iterations=1, num_context_points=MAX_CONTEXT_POINTS_NEW, num_target_points=MAX_TARGET_POINTS_NEW, batch_size = 1, use_data_augmentation="None", path_to_files=path_to_files,x_size=x_size,y_size=y_size, mode = "config", ratio_testing=0.,names_x=names_x, name_y=name_y)
    data_config = dataset_config.get_data(0, CONTEXT_IS_SUBSET)
    
    target_y_cnp_config = model(data_config.query, data_config.target_y)
    loss_config = bce(target_y_cnp_config,  data_train.target_y)

    target_y_cnp_config = target_y_cnp_config[0].detach().numpy()
    target_y = data_config.target_y[0].detach().numpy()
    df = pd.read_csv(file, index_col=0)
    tmp = df[["radius","thickness","npanels","theta","length"]].to_numpy()
    x         = np.append(x,[df[["radius","thickness","npanels","theta","length"]].to_numpy()[0]],axis=0)
    #x         = np.append(x,[data_config.query[1][0][0][-5:].numpy()],axis=0)

    sum_target_y_tmp = np.array([np.sum(target_y)])
    sum_target_y    = np.append(sum_target_y, [sum_target_y_tmp], axis=0)
    mean_tmp = np.array([np.mean(target_y_cnp_config)])
    mean_target_y_cnp = np.append(mean_target_y_cnp, [mean_tmp], axis=0)

    hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg = plotting.sum_hist(target_y_cnp_config, target_y, hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg)
    print("{}/{} {}, {}, radius: {} cm, test loss: {}".format(i,len(filelist),datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), mode,x[-1,0], loss_config))
    fout.write("{}, Iteration: {}, test loss: {}\n".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), it, loss_config))

    fig = plotting.plot_result_configwise(target_y_cnp_config, target_y, f'{loss_config:.2f}', x[-1])
    pdf.savefig(fig)
    plt.show()
    plt.clf()
        
fig1 = plotting.plot_result_summed(hist_target_sig, hist_target_bkg, hist_pred_sig, hist_pred_bkg)
pdf.savefig(fig1)
plt.show()
plt.clf()

fig2, ax = plt.subplots(ncols=2, figsize=(12, 4))
ax[0].plot(x[:,0],mean_target_y_cnp,'o')
ax[0].set_title('Conditional Neutral Process', fontsize=10)
ax[0].set_xlabel('radius [cm]')
#ax[0].set_ylim(0.0,0.15)
ax[1].plot(x[:,0],sum_target_y,'o')
ax[1].set_title('Simulation', fontsize=10)
ax[1].set_xlabel('radius [cm]')
pdf.savefig(fig2)
plt.show()
plt.clf()
pdf.close()
    
fout.close()

df = pd.DataFrame(x, columns=["Radius[cm]","Thickness[cm]","NPanels","Theta[deg]","Length[cm]"])
df['Ge-77[nevents]'] = sum_target_y
df['Ge-77_CNP[nevents]'] = mean_target_y_cnp
df=df.round(decimals=4)
df.to_csv(f'{path_out}Ge77_rates_CNP_{version}.csv')
